# Neural Network
This notebook applies a simple model to simple data.  
Data = Iris.  
Model = Fully Connected aka Dense aka Multi-Layer Perceptron (MLP).   
This notebook added added 5-fold cross validation to notebook 04. 

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2022-09-03 13:42:37.481769
Python 3.10.0


In [2]:
from tensorflow import keras
from keras import layers
from sklearn.model_selection import StratifiedKFold

In [3]:
from sklearn import datasets
from sklearn.utils import shuffle
iris = datasets.load_iris()
print('shape',iris.data.shape)
print('features',iris.feature_names)
print('labels',iris.target_names)
X = iris.data
y = iris.target
# At start, all species=0 come first and all species=2 come last.
# Important to shuffle rows before doing train/test split and train/valid split.
# Another way is using the sklearn train_test_split class.
X, y = shuffle(X, y, random_state=0)
print('X shape',X.shape)
print('y shape',y.shape)
num_instances = X.shape[0]
num_features =  X.shape[1]

shape (150, 4)
features ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
labels ['setosa' 'versicolor' 'virginica']
X shape (150, 4)
y shape (150,)


In [4]:
X

array([[5.8, 2.8, 5.1, 2.4],
       [6. , 2.2, 4. , 1. ],
       [5.5, 4.2, 1.4, 0.2],
       [7.3, 2.9, 6.3, 1.8],
       [5. , 3.4, 1.5, 0.2],
       [6.3, 3.3, 6. , 2.5],
       [5. , 3.5, 1.3, 0.3],
       [6.7, 3.1, 4.7, 1.5],
       [6.8, 2.8, 4.8, 1.4],
       [6.1, 2.8, 4. , 1.3],
       [6.1, 2.6, 5.6, 1.4],
       [6.4, 3.2, 4.5, 1.5],
       [6.1, 2.8, 4.7, 1.2],
       [6.5, 2.8, 4.6, 1.5],
       [6.1, 2.9, 4.7, 1.4],
       [4.9, 3.6, 1.4, 0.1],
       [6. , 2.9, 4.5, 1.5],
       [5.5, 2.6, 4.4, 1.2],
       [4.8, 3. , 1.4, 0.3],
       [5.4, 3.9, 1.3, 0.4],
       [5.6, 2.8, 4.9, 2. ],
       [5.6, 3. , 4.5, 1.5],
       [4.8, 3.4, 1.9, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [6.2, 2.8, 4.8, 1.8],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.8, 1.9, 0.4],
       [6.2, 2.9, 4.3, 1.3],
       [5. , 2.3, 3.3, 1. ],
       [5. , 3.4, 1.6, 0.4],
       [6.4, 3.1, 5.5, 1.8],
       [5.4, 3. , 4.5, 1.5],
       [5.2, 3.5, 1.5, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [6.4, 2

In [5]:
y

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0,
       0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1,
       1, 2, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 1, 1, 2, 2,
       1, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 2, 1, 2, 0])

Without normalization, the feature with biggest variance dominates.  
Unfortunately, on these data, normalization reduces the attainable accuracy.   
We use the Keras Normalization class:
1. Constructor takes axis, mean, var. By default, axis = -1 (the last axis), mean=0, var=1.
1. The adapt() method tunes the scaling parameters to the given data.
1. From then on, the object scales any given data.
1. Importantly, we adapt() to the training data and apply to the test data.

In [6]:
# Normalize on half the data. This isn't exactly fair, as some test data is included.
X_train = X[:75] 
normalizer = layers.Normalization() 
normalizer.adapt(X_train)
X = normalizer(X)
X

2022-09-03 13:42:49.193765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(150, 4), dtype=float32, numpy=
array([[-0.04237568, -0.4948195 ,  0.7840797 ,  1.7010827 ],
       [ 0.23398574, -1.8361979 ,  0.09298381, -0.33556128],
       [-0.45691848,  2.6350634 , -1.5405157 , -1.4993577 ],
       [ 2.0303369 , -0.2712561 ,  1.5380026 ,  0.82823515],
       [-1.1478227 ,  0.84655935, -1.4776887 , -1.4993577 ],
       [ 0.6485285 ,  0.622996  ,  1.3495218 ,  1.8465571 ],
       [-1.1478227 ,  1.0701222 , -1.6033425 , -1.3538831 ],
       [ 1.2012514 ,  0.17586967,  0.532772  ,  0.39181152],
       [ 1.3394327 , -0.4948195 ,  0.5955991 ,  0.24633694],
       [ 0.37216645, -0.4948195 ,  0.09298381,  0.10086234],
       [ 0.37216645, -0.9419458 ,  1.0982141 ,  0.24633694],
       [ 0.78670925,  0.39943308,  0.40711832,  0.39181152],
       [ 0.37216645, -0.4948195 ,  0.532772  , -0.04461209],
       [ 0.92489   , -0.4948195 ,  0.46994513,  0.39181152],
       [ 0.37216645, -0.2712561 ,  0.532772  ,  0.24633694],
       [-1.2860034 ,  1.2936851 , -

Keras offers 2 means of building a model.  
1. The [Sequential class](https://keras.io/api/models/sequential/#sequential-class) is simple. It has an add(layer) method. It maintains layers in the order they were added. It allows linear models only. Every layer has one input and one output.
1. The [Functional API](https://keras.io/guides/functional_api/) is complex. It allows DAG models. Layers can be skipped, reused, and multiply connected.

The Dense layer has these constructor options.
1. num units
1. activation. Default=linear. Choices are relu, elu, softplus (smooth relu), softmax, sigmoid, tanh, exponential. There is an API so you can build your own.
1. use_bias. Default=True
1. kernel_initializer. Default="glorot_uniform"
1. regularizers for the bias, kernel, and activity. Default=None.

In [7]:
inputs = keras.Input(shape=(num_features,))
inputs

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'input_1')>

In [8]:
# Keras output layer raises exception unless #neurons == #labels in Y.
dense1 =  layers.Dense(8, activation="relu")   (inputs)
dense2 =  layers.Dense(8, activation="relu")   (dense1)
outputs = layers.Dense(3)                      (dense2)
outputs

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'dense_2')>

In [9]:
model = keras.Model(inputs=inputs, outputs=outputs, name="MLP_1")
model.summary()

Model: "MLP_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 8)                 40        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 3)                 27        
                                                                 
Total params: 139
Trainable params: 139
Non-trainable params: 0
_________________________________________________________________


Explain the number of trainable parameters:
1. Dense_0: (4 input + 1 bias) * 8 out = 40 
1. Dense_0: (8 input + 1 bias) * 8 out = 72 
1. Dense_2: (8 input + 1 bias) * 3 out = 27

In [10]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

Note on model.fit() and model.evaluate():
Both methods are written to be customized.
By default, [model.fit()](https://keras.io/api/models/model_training_apis/#fit-method)
has shuffle=True within each batch.  

In [20]:
EPOCHS=100
fold=1
skf = StratifiedKFold(n_splits=5)
history_vector=[]
test_vector=[]
for train, test in skf.split(X,y):
    print(type(X))
    print(type(train))
    X_train = np.array(X)[train]
    y_train = np.array(y)[train]
    X_test  = np.array(X)[test]
    y_test  = np.array(y)[test]
    model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),metrics=["accuracy"])
    history = model.fit(X_train, y_train, batch_size=25, epochs=EPOCHS)
    test_scores = model.evaluate(X_test, y_test)
    print("Fold", fold, "Test loss:", test_scores[0], "Test accuracy:", test_scores[1])
    fold += 1
    history_vector.append(history)
    test_vector.append(test_scores)

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
Epoch 1/100
5/5 [==============================] - 1s 4ms/step - loss: 0.0584 - accuracy: 0.9750
Epoch 2/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0537 - accuracy: 0.9667
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0532 - accuracy: 0.9833
Epoch 4/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0533 - accuracy: 0.9750
Epoch 5/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0528 - accuracy: 0.9750
Epoch 6/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0526 - accuracy: 0.9833
Epoch 7/100
5/5 [==============================] - 0s 6ms/step - loss: 0.0525 - accuracy: 0.9750
Epoch 8/100
5/5 [==============================] - 0s 11ms/step - loss: 0.0512 - accuracy: 0.9833
Epoch 9/100
5/5 [==============================] - 0s 7ms/step - loss: 0.0518 - accuracy: 0.9833
Epoch 10/100
5/5 [==============================

5/5 [==============================] - 0s 4ms/step - loss: 0.0456 - accuracy: 0.9833
Epoch 84/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0461 - accuracy: 0.9833
Epoch 85/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0459 - accuracy: 0.9833
Epoch 86/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0451 - accuracy: 0.9833
Epoch 87/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0460 - accuracy: 0.9750
Epoch 88/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0451 - accuracy: 0.9833
Epoch 89/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0464 - accuracy: 0.9833
Epoch 90/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0459 - accuracy: 0.9833
Epoch 91/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0453 - accuracy: 0.9750
Epoch 92/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0453 - accuracy: 0.9833
Epoch 93/100
5/5 [===============

5/5 [==============================] - 0s 3ms/step - loss: 0.0160 - accuracy: 1.0000
Epoch 65/100
5/5 [==============================] - 0s 6ms/step - loss: 0.0156 - accuracy: 1.0000
Epoch 66/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0156 - accuracy: 1.0000
Epoch 67/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0157 - accuracy: 1.0000
Epoch 68/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 69/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 70/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 71/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0151 - accuracy: 1.0000
Epoch 72/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 73/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 74/100
5/5 [===============

5/5 [==============================] - 0s 3ms/step - loss: 0.0441 - accuracy: 0.9833
Epoch 46/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0460 - accuracy: 0.9833
Epoch 47/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0443 - accuracy: 0.9833
Epoch 48/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0439 - accuracy: 0.9833
Epoch 49/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0459 - accuracy: 0.9833
Epoch 50/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0450 - accuracy: 0.9833
Epoch 51/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0444 - accuracy: 0.9833
Epoch 52/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0452 - accuracy: 0.9833
Epoch 53/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0443 - accuracy: 0.9833
Epoch 54/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0442 - accuracy: 0.9833
Epoch 55/100
5/5 [===============

5/5 [==============================] - 0s 5ms/step - loss: 0.0423 - accuracy: 0.9833
Epoch 27/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0424 - accuracy: 0.9833
Epoch 28/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0422 - accuracy: 0.9833
Epoch 29/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0424 - accuracy: 0.9833
Epoch 30/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0418 - accuracy: 0.9833
Epoch 31/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0425 - accuracy: 0.9833
Epoch 32/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0419 - accuracy: 0.9833
Epoch 33/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0428 - accuracy: 0.9833
Epoch 34/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0421 - accuracy: 0.9833
Epoch 35/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0415 - accuracy: 0.9833
Epoch 36/100
5/5 [===============

Epoch 1/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0395 - accuracy: 0.9917
Epoch 2/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0380 - accuracy: 0.9917
Epoch 3/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0367 - accuracy: 0.9833
Epoch 4/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0368 - accuracy: 0.9833
Epoch 5/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0357 - accuracy: 0.9917
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0360 - accuracy: 0.9917
Epoch 7/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0372 - accuracy: 0.9917
Epoch 8/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0360 - accuracy: 0.9833
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0356 - accuracy: 0.9917
Epoch 10/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0355 - accuracy: 0.9917
Epoch 11/100
5/5 [===========

5/5 [==============================] - 0s 4ms/step - loss: 0.0338 - accuracy: 0.9833
Epoch 85/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0340 - accuracy: 0.9917
Epoch 86/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0329 - accuracy: 0.9917
Epoch 87/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0343 - accuracy: 0.9917
Epoch 88/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0344 - accuracy: 0.9917
Epoch 89/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0337 - accuracy: 0.9917
Epoch 90/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0337 - accuracy: 0.9917
Epoch 91/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0329 - accuracy: 0.9917
Epoch 92/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0342 - accuracy: 0.9833
Epoch 93/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0340 - accuracy: 0.9833
Epoch 94/100
5/5 [===============

In [22]:
for pair in test_vector:
    print('Test acc =',pair[1],'Test loss =',pair[0])

Test acc = 1.0 Test loss = 0.03155630826950073
Test acc = 0.9333333373069763 Test loss = 0.14157238602638245
Test acc = 1.0 Test loss = 0.0011016553035005927
Test acc = 1.0 Test loss = 0.012707909569144249
Test acc = 0.9666666388511658 Test loss = 0.053446076810359955
